In [18]:
import numpy
import gym
import tensorflow as tf
from tensorflow.contrib.layers import flatten, conv2d, fully_connected
from collections import deque, Counter
import random
from datetime import datetime

In [ ]:
env = gym.make('MsPacman-v0')
n_outputs = env.action_space.n
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample())

color = np.array([210,164,74]).mean()

def preprocess_observation(obs):
    img = obs[1:176:2, ::2]
    imh = img.mean(axis=2)
    img[img==color] = 0
    img = (img - 128) / 128 - 1
    return img.reshape(88,80,1)

tf.reset_default_graph()

def q_network(X, name_scope):
    initilizer = tf.contrib.layers.variance_scaling_initializer()
    
    with tf.variable_scope(name_scope) as scope:
        
        layer_1 = conv2d(X, num_outputs=32, kernel_size=(8,8), stride=4, padding='SAME', 
                         weights_initilizer=initilizer)
        tf.summary.histogram('layer_1', layer_1)
        
        layer_2 = conv2d(layer_1, num_outputs=64, kernel_size=(4,4), stride=2, padding='SAME',
                        weights_initilizer=initilizer)
        tf.summary.histogram('layer_2', layer_2)
        
        layer_3 = conv2d(layer_2, num_outputs=64, kernel_size=(3,3), stride=1, padding='SAME',
                        weights_initilizer=initilizer)
        tf.summary.histogram('layer_3', layer_3)
        
        flat = flatten(layer_3)
        
        fc = fully_connected(flat, num_outputs=128, weights_initilizer=initilizer)
        tf.summary.histogram('fc', fc)
        
        output = fully_connected(fc, num_outputs=n_outputs, activation_fn=None,
                                weights_initilizer=initilizer)
        tf.summary.histogram('output', output)
        
        
        vars = {v.name[len(scope.name):]: v for v in
               tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope.name)}        
    return vars, output



    

        

    
        
        